Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
import re

In [2]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('./whiskey-reviews-dspt4/train.csv')
test = pd.read_csv('./whiskey-reviews-dspt4/test.csv')
print(train.shape, test.shape)

(4087, 3) (1022, 2)


In [3]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [99]:
def clean_data(X):
    X = X.copy()
    X['description'] = X['description'].apply(lambda text: text.replace("\n", ""))
    
    return X

In [100]:
train = clean_data(train)
train.head()

,id,description,ratingCategory
0,1321,"Sometimes, when whisky is batched, a few leftover barrels are returned to the warehouse. Canadian Club recently pulled and vatted several of these from the 1970s. Acetone, Granny Smith apples, and fresh-cut white cedar showcase this long age. Complex and spicy, yet reserved, this dram is ripe with strawberries, canned pears, cloves, pepper, and faint flowers, then slightly pulling oak tannins. Distinct, elegant, and remarkably vibrant, this ancient Canadian Club is anything but tired. (Australia only) A$133",1
1,3861,"An uncommon exclusive bottling of a 6 year old cask strength malt. Light gold in color, the nose is vegetal, more peat bog than peat smoke, with an undercurrent of pastry cream and rose. It’s an odd combination of aromas. The entry is flavorful and inviting with smoked pineapple, clove, and rose. Peak smoke arrives in full force in the mid-palate, which drops the sweet and becomes spicy. The finish is mostly smoke, but with a pleasant minty coolness. (Wyoming only)",0
2,655,"This release is a port version of Amrut’s Intermediate Sherry — a sort of port pipe sandwich. The spirit is matured in both unused casks and bourbon casks, then spends a few months in port pipes, and then returns to bourbon casks. The result is a Pink Floyd show of a whisky: vibrant, colorful, complex, and nearly too much. A blackcurrant and wispy, smoky nose gives way to an intense and bittersweet mix of chili, blackcurrant, oak, damson, dark chocolate, and peat. Astounding.",1
3,555,"This 41 year old single cask was aged in a sherry butt and interacted magnificently with that wood. The nose offers peeled red apples, sultanas, honey, fudge, milk chocolate, and American cream soda. Balanced and harmonious throughout, the palate yields remarkably fresh orchard fruits, pipe tobacco, black pepper, and ginger. Long in the finish, with cinnamon drying to dark chocolate, and slightly smoky, unobtrusively tannic oak. A 1970s classic!",1
4,1965,"Quite herbal on the nose, with aromas of dried tarragon, parsley, and dill, along with cherry, milk chocolate, orange blossom, vanilla bean, and sweet tea. The palate is slightly grain-forward, but is full and well balanced, with orange, dried flowers, chamomile tea, lemon rind, and ginger. The finish offers cinnamon and milk chocolate against a note of candied lemon. Youthful and barrel-influenced, but admirably executed.",1


In [101]:
test = clean_data(test)

In [64]:
# Distribution of ratingCategory: 0 (Excellent), 1 (Good), 2 (Poor)
train.ratingCategory.value_counts()

1    2881
0    1141
2    65  
Name: ratingCategory, dtype: int64

In [65]:
# Read a few reviews from the "Excellent" category
pd.set_option('display.max_colwidth', 0)
train[train.ratingCategory == 0].sample(3)

,id,description,ratingCategory
2652,4820,"The Eigashima distillery, on the Akashi Strait near Kobe, may be the least well known of Japan’s single malt plants, but has a sound claim to be the country’s oldest, as its license to make whisky was granted in 1919 — four years before Yamazaki was built. It has, however, specialized in shochu, and even now only turns its hand to whisky making for two months every year. This 5 year old — bottled for independent Japanese specialist Number One Drinks — represents a tentative move into the single malt market once more. Pale in color, it shows a typically Japanese cleanliness on the nose that’s cut with a touch of waxiness. It opens with a scented angelica-like lift, there’s even a whiff of something like gooseberry jam. As it opens, the aroma darkens slightly, showing a touch of roasted tea. Water brings out a little yeastiness (typical for some younger whiskies), alongside cucumber, borage, and lime. The palate is sweet with vanilla custard and a sweet, ginger-accented note leading to ripe pear.\rA charming malt, and already well-balanced for its age. Here’s Japan at its lightest. £55. Price in US dollars converted at time of review.",0
3027,4784,"Pale straw. The nose is very draff-like/sweet mash, then green herbal notes, reminiscent of angelica. The palate remains light and slightly hot on the tongue, with some cereal and a certain fatness. Water brings out a little more from the cask. A delicate and pleasant enough dram, but I’m slightly bemused as to why it’s being bottled at this age when there’s clearly so much more to give. £69",0
2128,4253,"First impression is of molten Reese’s peanut butter cups. This 13 year old single grain whiskey has been finished in second-fill red wine casks from Ànima Negra in Mallorca, however this wine cask has given the whiskey a kick in the Balearics. The grain character takes a drubbing, the nose engulfed by jam pots and loganberry. It is gentle and deliciously fruity, reminiscent of strawberry, raspberry pop tarts, and citrus groves. Black pepper and caramelized sugar to close. (417 bottles) €90",0


In [6]:
# Read a few reviews from the "Poor" category
train[train.ratingCategory == 2].sample(3)

,id,description,ratingCategory
3319,5084,"\nMaple syrup and gelatin capsules in a veterinarian’s office. Texas Spirit is WB Liquors’ (Costco) house brand and so, available in one size only: extra-large. A simple mixing whisky, it is sweet, floral, watery, and loaded with vanilla and caramel. Extra hot pepper throws it a bit out of balance, though it would likely go well with Coke. A young, unassuming whisky not to savor, but to session. Price is per 1.75 liter.",2
1844,5065,"\nShowing its youth through an obvious green character that peers from beneath a cloak of campfire and applewood smoke, with banana as the prominent fruit, along with secondary apple, citrus, and a peculiar rubber tire note. The palate is light-bodied and overtly sweet, with grass, marshmallow, and vanilla. Reminiscent of a Lowland malt, which only contributes to the disjointed feeling of the whole package.",2
1667,5081,"\nAged for “at least” one month, this bourbon is a collaboration with the band Fierce Dead Rabbit. Better Days is pale gold and noticeably cloudy. On the nose it’s paste, yeasty bread dough, and wet pavement. On the palate it is all over the place with raw oak, cinnamon, almond, and black pepper. There's no balance and no integration. The finish is short, hot, and dry. As whiskey ages, it goes through odd, awkward phases, and that's where this one is.",2


### Split the Training Set into Train/Validation

In [67]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train['description'], 
                                                    train['ratingCategory'], 
                                                    test_size=0.2, 
                                                    stratify=train['ratingCategory'],
                                                    random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3269,) (818,) (3269,) (818,)


### Define Pipeline Components

In [8]:
from sklearn.pipeline import Pipeline

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('rfc', rfc)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [9]:
X_train.head()

1782    \nVery fragrant, with notes of fresh ripe plum, crisp spice, herbs, caramel, treacle, black pepper, and hints of dark chocolate. Leave the glass alone and the spices really hit their stride. The flavor shifts imperceptibly, covering citrus, vanilla, chocolate, and ripe orchard fruits, generating a slow build-up of spices like a gathering storm: your taste buds have nowhere left to run. Beautifully paced.                                                                             
3414    \nThe good news: This is one of the best Highland Park whiskies I have ever tasted. The bad news: it’s the new release for Travel Retail (formerly Duty Free). It’s lush, well-balanced, and very complex. Well-defined notes of toffee, candied fruit, and roasted nuts are accentuated by background honey, chocolate mousse, and smoke. Here’s the icing on the cake: it’s bottled at 47.5%, which really allows the flavors to shine. If you’re going overseas, consider tracking down a bottle.
2744    \nA sw

In [10]:
parameters = {
    'vect__max_df': (0.7, 1.0),
    'vect__min_df': (2, 5, 10),
    'rfc__n_estimators': (100, 500),
    'rfc__max_depth': (10, 20)
}

grid_search = GridSearchCV(pipe, parameters, cv=4, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 4 folds for each of 24 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:  2.7min finished


GridSearchCV(cv=4, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [11]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.721626021612854
{'rfc__max_depth': 20, 'rfc__n_estimators': 100, 'vect__max_df': 1.0, 'vect__min_df': 10}


In [12]:
from sklearn.metrics import accuracy_score

# Evaluate on test data
score = grid_search.predict(X_test)
accuracy_score(score, y_test)

0.7188264058679706

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [19]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [20]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [21]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [22]:
subNumber = 0

In [23]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./whiskey-reviews-dspt4/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [24]:
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import TruncatedSVD

# Use parameters from GridSearch in previous section
vect = TfidfVectorizer(stop_words='english', 
                       ngram_range=(1,2),
                       min_df=10, 
                       max_df=1.0)

# Add dimensionality reduction
svd = TruncatedSVD(algorithm='randomized',
                   n_iter=10)

# Use parameters from RandomSearch in previous section
rfc = RandomForestClassifier(n_estimators=500,
                             max_depth=20,
                             random_state=42)


In [25]:
pipe = Pipeline([
    ('vect', vect),      # TF-IDF Vectorizer
    ('svd', svd),        # Truncated SVD Dimensionality Reduction
    ('rfc', rfc)         # RandomForest Classifier
])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [30]:
params = {
    'svd__n_components': stats.randint(100, 500),
    'vect__max_df': (0.7, 1.0),
    'vect__min_df': (2, 5, 10),
    'rfc__n_estimators': (100, 500),
    'rfc__max_depth': (10, 20)
}

In [31]:
# Fit
random_search = RandomizedSearchCV(pipe, params, cv=2, n_iter=3, n_jobs=-1, verbose=1)
random_search.fit(X_train, y_train)

Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   19.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   19.6s finished


RandomizedSearchCV(cv=2, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('vect',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                                              min_df=10,
                                                       

In [32]:
print(random_search.best_score_)
print(random_search.best_params_)

0.70755617441299
{'rfc__max_depth': 10, 'rfc__n_estimators': 500, 'svd__n_components': 182, 'vect__max_df': 1.0, 'vect__min_df': 5}


In [33]:
# Evaluate on test data
score = random_search.predict(X_test)
accuracy_score(score, y_test)

0.7139364303178484

### Make a Submission File

In [34]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [35]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [36]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [37]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./whiskey-reviews-dspt4/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [68]:
# Apply to your Dataset
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint

param_dist = {
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [39]:
# Continue Word Embedding Work Here
import spacy
nlp = spacy.load("en_core_web_lg")

In [40]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [69]:
X = get_word_vectors(X_train)
len(X) == len(X_train)

True

In [70]:
X_test_vec = get_word_vectors(X_test)

In [71]:
rfc.fit(X, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=20, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [72]:
score = rfc.predict(X_test_vec)
accuracy_score(score, y_test)

0.7432762836185819

### Make a Submission File

In [73]:
# Predictions on test sample
pred = rfc.predict(get_word_vectors(test['description']))

In [74]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [75]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [76]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv(f'./whiskey-reviews-dspt4/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
        - **Sentiment Analysis is the interpretation and classification of emotions within text data.**
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
        - **Documnet Classification analyzes what the overall topic of the docment is while Sentiment Analysis
          picks up on the overall emotion of the document whether that be positive, negative, or nutral.**
    - How do create labeled sentiment data? Are those labels really sentiment?
        - **One way is to do it manually, one suggestion I found was to start with the easy ones, then make
          a model off that and use it to help with the more difficult ones later to get the model's "opinion".**
    - What are common applications of sentiment analysis?
        - **Companies can use sentiment analysis on comments to find out if people are generally positive or 
          negative about their service or product.**
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?